In [1]:
%matplotlib inline

import gc
import os
import sys
import pandas as pd
import numpy as np
import tqdm
import seaborn as sns
from scipy.stats import mode
import pickle

from sklearn.model_selection import cross_val_score
from sklearn.metrics import recall_score, precision_score
import platform
from collections import Counter
from sklearn.metrics import recall_score, matthews_corrcoef
from lightgbm import LGBMClassifier, LGBMRegressor, log_evaluation
from lightgbm import early_stopping
from sklearn.linear_model import LogisticRegression

import matplotlib
import matplotlib.pyplot as plt 

from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import roc_auc_score, matthews_corrcoef, accuracy_score
from sklearn.metrics import recall_score, precision_score

#from utils import prepare_df, transform_datetime_features

pd.set_option('display.max_columns', None)

# если у вас есть CUDA, то она понадобится там для экспериментов в catboost
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

task_type = 'GPU'
if platform.node() == 'VLAD2016':
    task_type = 'CPU'
    
import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)
warnings.simplefilter(action = 'ignore', category = DeprecationWarning)
warnings.simplefilter(action = 'ignore', category = UserWarning)
warnings.simplefilter(action = 'ignore', category = RuntimeWarning)
warnings.filterwarnings("ignore", message = "numpy.dtype size changed")
warnings.filterwarnings("ignore", message = "numpy.ufunc size changed")
pd.options.mode.chained_assignment = None

DEBUG = False
DEBUG = False
target = 'time_to_under'
target2 = 'label'

In [2]:
dtrain = pd.read_csv('../input/train_dataset_train.csv')
dtrain.head(3)

,id,ticket_id,ticket_type_nm,entrance_id,entrance_nm,station_id,station_nm,line_id,line_nm,pass_dttm,time_to_under,label
0,1,F94EDC107A4E054786882F9453C1B93F,Пропуск FacePay,184,Молодёжная (Западный),3010.0,Молодёжная,3.0,Арбатско-Покровская,2022-09-12 05:00:45,475.050000,3010
1,2,8F13BFBA98A7DD34B11BD2DDCC5CA68E,СК студента 30 дней,110,Войковская ( Южный ),2006.0,Войковская,2.0,Замоскворецкая,2022-09-12 05:00:52,647.216667,2006
2,3,126727A96489CC976A8C08E5CEB00542,СК учащегося 30 дней,110,Войковская ( Южный ),2006.0,Войковская,2.0,Замоскворецкая,2022-09-12 05:00:54,648.183333,2006


In [3]:
dtest = pd.read_csv('../input/test_dataset_test.csv')
dtest.head(3)

,id,ticket_id,ticket_type_nm,entrance_id,entrance_nm,station_id,station_nm,line_id,line_nm,pass_dttm
0,1091022,425A13E16226CA01032043111488E0F5,КОШЕЛЕК,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:00:03
1,1091023,F94EDC107A4E054786882F9453C1B93F,Пропуск FacePay,184,Молодёжная (Западный),3010,Молодёжная,3,Арбатско-Покровская,2022-09-12 05:00:45
2,1091024,8F13BFBA98A7DD34B11BD2DDCC5CA68E,СК студента 30 дней,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:00:52


In [4]:
use = [f for f in dtrain.columns if f not in ['id', 'ticket_id', 'pass_dttm', target, target2]]

In [5]:
numerics = ['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerical_columns = []
categorical_columns = []
remove = []
dtypes = {}

for c in use:
    if dtrain[c].dtype == 'object':
        print('########')
        print('categorical_columns', c)
        categorical_columns.append(c)
        d = {c: 'category'}
        dtypes.update(d)
        print(len(dtrain[c].unique()))
        print(dtrain[c].value_counts(dropna = False) / len(dtrain)*100)
        print('########')
        
    elif dtrain[c].dtypes in numerics:
        numerical_columns.append(c)
        #d = {c: str(t).split("'")[0]}
        #dtypes.update(d)
        
    else:
        print('unknow type', c)
        
    if len(dtrain[c].value_counts(dropna = True)) == 1:
        print('remove', c)
        remove.append(c)
        
    if len(dtrain[c].unique()) == dtrain.shape[0]:
        print('remove', c)
        remove.append(c)

########
categorical_columns ticket_type_nm
59
КОШЕЛЕК                                                   23.997936
30 дней ЕДИНЫЙ ТК                                         16.294982
Социальная карта москвича                                 11.597831
30 дней                                                   10.733229
90 дней ЕДИНЫЙ ТК                                          5.303371
30 дней СК студента                                        4.465625
60 поездок ЕДИНЫЙ ТК                                       4.288910
30 дней Пригород ТК                                        4.127411
СК студента 30 дней                                        3.638695
ББК                                                        2.700218
30 дней СК учащегося                                       2.210952
365 дней ЕДИНЫЙ ТК                                         1.931765
СК учащегося 30 дней                                       1.587684
Социальная карта жителя Моск. области                      1.410328
9

In [6]:
for df in [dtrain, dtest]:
    df.loc[df['ticket_type_nm'] == 'Билет 1 сутки ЕДИНЫЙ','ticket_type_nm'] = 'otrher'
    df.loc[df['ticket_type_nm'] == 'ВЛБ МОСКВА','ticket_type_nm'] = 'otrher'
    df.loc[df['ticket_type_nm'] == 'ВЕСБ МО (тип 6)','ticket_type_nm'] = 'otrher'

In [7]:
numerics = ['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerical_columns = []
categorical_columns = []
remove = []
dtypes = {}

for c in use:
    if dtrain[c].dtype == 'object':
        print('########')
        print('categorical_columns', c)
        categorical_columns.append(c)
        d = {c: 'category'}
        dtypes.update(d)
        print(len(dtrain[c].unique()))
        print(dtrain[c].value_counts(dropna = False) / len(dtrain)*100)
        print('########')
        
    elif dtrain[c].dtypes in numerics:
        numerical_columns.append(c)
        #d = {c: str(t).split("'")[0]}
        #dtypes.update(d)
        
    else:
        print('unknow type', c)
        
    if len(dtrain[c].value_counts(dropna = True)) == 1:
        print('remove', c)
        remove.append(c)
        
    if len(dtrain[c].unique()) == dtrain.shape[0]:
        print('remove', c)
        remove.append(c)

########
categorical_columns ticket_type_nm
57
КОШЕЛЕК                                                   23.997936
30 дней ЕДИНЫЙ ТК                                         16.294982
Социальная карта москвича                                 11.597831
30 дней                                                   10.733229
90 дней ЕДИНЫЙ ТК                                          5.303371
30 дней СК студента                                        4.465625
60 поездок ЕДИНЫЙ ТК                                       4.288910
30 дней Пригород ТК                                        4.127411
СК студента 30 дней                                        3.638695
ББК                                                        2.700218
30 дней СК учащегося                                       2.210952
365 дней ЕДИНЫЙ ТК                                         1.931765
СК учащегося 30 дней                                       1.587684
Социальная карта жителя Моск. области                      1.410328
9

In [8]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

enc = OneHotEncoder(handle_unknown = 'ignore')
le  = LabelEncoder()
L = dtrain.shape[0]

for u in categorical_columns:
    print(u, len(dtrain[u].unique()))
    
    if len(dtrain[u].unique()) < 100:
        df = pd.DataFrame(pd.concat([dtrain[u], dtest[u]], axis = 0))
        df[u] = df[u].fillna('none')
        df[u] = df[u].apply(lambda x: str(x).lower())
        df[u] = df[u].astype(str)
        temp = enc.fit_transform(df[u].values.reshape(-1, 1)).toarray()
        for i in range(temp.shape[1]):
            n = u + '_' + str(i)
            dtrain[n] = temp[:, i][:L]
            dtest[n] = temp[:, i][L:]
            if n not in use:
                use.append(n)        
        if u in use:
            use.remove(u)       
    
        dtrain.drop(u, axis = 1, inplace = True)
        dtest.drop(u, axis = 1, inplace = True)
    
    else:
        df = pd.DataFrame(pd.concat([dtrain[u], dtest[u]], axis = 0))
        df[u] = df[u].fillna('none')
        df[u] = df[u].astype(str)
        le.fit(df[u].values.ravel())
        temp = le.transform(df[u].values.ravel())
        dtrain[u] = temp[:L]
        dtest[u]  = temp[L:]        
    
gc.collect()

ticket_type_nm 57
entrance_nm 427
station_nm 245
line_nm 16


0

In [9]:
for df in [dtrain, dtest]:

    df["day"] = df.pass_dttm.map(lambda x: int(x.split("-")[2].split(" ")[0]))
    df["hours"] = df.pass_dttm.map(lambda x: int(x.split("-")[2].split(" ")[1].split(":")[0]))

In [10]:
dtrain["pass_dttm"] = pd.to_datetime(dtrain["pass_dttm"])
dtest["pass_dttm"] = pd.to_datetime(dtest["pass_dttm"])

In [11]:
u = 'pass_dttm'

for df in [dtrain, dtest]:
    
    df['number_weekday'] = df[u].dt.weekday
    df['number_month'] = df[u].dt.month
    df['number_day'] = df[u].dt.day

    df['number_hour'] = df[u].dt.hour
    df['number_hour_of_week'] = df[u].dt.hour + df[u].dt.weekday * 24
    df['number_minute_of_day'] = df[u].dt.minute + df[u].dt.hour * 60

    #number_minute_of_day
    min_in_day = 24 * 60
    df['sin_time'] = np.sin(2 * np.pi * df['number_minute_of_day'] / min_in_day)
    df['cos_time'] = np.cos(2 * np.pi * df['number_minute_of_day'] / min_in_day)
            
    #number_hour_of_day
    hour_in_day = 24
    df['sin_hour'] = np.sin(2 * np.pi * df['number_hour']  / hour_in_day)
    df['cos_hour'] = np.cos(2 * np.pi * df['number_hour']  / hour_in_day)
              
    #number_day_of_mont
    day_in_month = 31
    df['sin_day'] = np.sin(2 * np.pi * df['number_day'] / day_in_month)
    df['cos_day'] = np.cos(2 * np.pi * df['number_day'] / day_in_month)
        
    #number_month_of_year
    month_in_year = 12
    df['sin_month'] = np.sin(2 * np.pi * df['number_month'] / month_in_year)
    df['cos_month'] = np.cos(2 * np.pi * df['number_month'] / month_in_year)
        
    #number_weekday_of_week
    day_in_week = 7
    df['sin_weekday'] = np.sin(2 * np.pi * df['number_weekday'] / day_in_week)
    df['cos_weekday'] = np.cos(2 * np.pi * df['number_weekday'] / day_in_week)

In [12]:
use = [f for f in dtrain.columns if f not in ['id', 'ticket_id', 'pass_dttm', target, target2]]
use

['entrance_id',
 'entrance_nm',
 'station_id',
 'station_nm',
 'line_id',
 'ticket_type_nm_0',
 'ticket_type_nm_1',
 'ticket_type_nm_2',
 'ticket_type_nm_3',
 'ticket_type_nm_4',
 'ticket_type_nm_5',
 'ticket_type_nm_6',
 'ticket_type_nm_7',
 'ticket_type_nm_8',
 'ticket_type_nm_9',
 'ticket_type_nm_10',
 'ticket_type_nm_11',
 'ticket_type_nm_12',
 'ticket_type_nm_13',
 'ticket_type_nm_14',
 'ticket_type_nm_15',
 'ticket_type_nm_16',
 'ticket_type_nm_17',
 'ticket_type_nm_18',
 'ticket_type_nm_19',
 'ticket_type_nm_20',
 'ticket_type_nm_21',
 'ticket_type_nm_22',
 'ticket_type_nm_23',
 'ticket_type_nm_24',
 'ticket_type_nm_25',
 'ticket_type_nm_26',
 'ticket_type_nm_27',
 'ticket_type_nm_28',
 'ticket_type_nm_29',
 'ticket_type_nm_30',
 'ticket_type_nm_31',
 'ticket_type_nm_32',
 'ticket_type_nm_33',
 'ticket_type_nm_34',
 'ticket_type_nm_35',
 'ticket_type_nm_36',
 'ticket_type_nm_37',
 'ticket_type_nm_38',
 'ticket_type_nm_39',
 'ticket_type_nm_40',
 'ticket_type_nm_41',
 'ticket_typ

In [13]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
dtrain['ticket_id'] = le.fit_transform(dtrain['ticket_id'].values)

In [14]:
dtrain['number_hour'].unique()

array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
       22, 23,  0,  1,  2,  3,  4], dtype=int64)

In [15]:
dtest['number_hour'].unique()

array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
       22, 23,  0,  1,  2,  3,  4], dtype=int64)

In [ ]:
feats = []
dtemp = dtrain[['ticket_id', 'number_hour']]
dtemp_index = dtrain.ticket_id.value_counts().index

for u in tqdm.tqdm(dtemp_index):
    for d in dtemp.number_hour.unique():
        c = dtemp[(dtemp['ticket_id'] == u) & (dtemp['number_hour'] == d)].shape[0]        
        f = str(d) + '_count_hour'
        if f not in dtrain.columns:
            dtrain[f] = 0        
            dtrain.loc[(dtrain['ticket_id'] == u) & (dtrain['number_hour'] == d), f] = c
        else:
            dtrain.loc[(dtrain['ticket_id'] == u) & (dtrain['number_hour'] == d), f] = c
            
        if f not in feats:
            feats.append(f)
        
    dtemp = dtemp.loc[~(dtemp['ticket_id'] == u)]
    gc.collect()

  0%|                                                                          | 352/335533 [02:56<47:21:53,  1.97it/s]

In [ ]:
dtrain[feats]

In [ ]:
dtrain[feats].min(), dtrain[feats].max()

In [ ]:
feats = []
dtemp = dtrain[['ticket_id', 'number_weekday']]
dtemp_index = dtrain.ticket_id.value_counts().index

for u in tqdm.tqdm(dtemp_index):
    for d in dtemp.number_weekday.unique():
        c = dtemp[(dtemp['ticket_id'] == u) & (dtemp['number_weekday'] == d)].shape[0]        
        f = str(d) + '_count'
        if f not in dtrain.columns:
            dtrain[f] = 0        
            dtrain.loc[(dtrain['ticket_id'] == u) & (dtrain['number_weekday'] == d), f] = c
        else:
            dtrain.loc[(dtrain['ticket_id'] == u) & (dtrain['number_weekday'] == d), f] = c
            
        if f not in feats:
            feats.append(f)
        
    dtemp = dtemp.loc[~(dtemp['ticket_id'] == u)]
    gc.collect()

In [ ]:
dtrain[feats]

In [ ]:
dtrain[feats].min(), dtrain[feats].max()

In [ ]:
dtrain['ticket_id'] = le.inverse_transform(dtrain['ticket_id'].values)

In [ ]:
with open('dtrain.pkl', 'wb') as f:
    pickle.dump(dtrain, f)